In [ ]:
import pandas as pd
import plotly.express as px

file_path = 'data/dev_tr_survey_2023.csv'

df = pd.read_csv(file_path)
for column in df.columns:
    if column not in  ['Zaman damgası', 'Ağırlıklı olarak hangi dil ve teknolojileri kullanıyorsunuz?','Yan haklarınız neler?']:
        column_count = df[column].value_counts()
        column_df = pd.DataFrame({column: column_count.index, 'Count': column_count.values})
        fig = px.pie(column_df, values='Count', names=column)
        fig.update_layout(title_text=column)
        fig.show()
    elif column in ['Ağırlıklı olarak hangi dil ve teknolojileri kullanıyorsunuz?','Yan haklarınız neler?']:
        df[column] = df[column].str.replace('doğum günü izni, regl izni gibi', 'doğum günü izni|regl izni gibi')
        df_positions = df[column].str.split(', ').explode()
        position_counts = df_positions.value_counts()
        fig = px.bar(position_counts, x=position_counts.index, y=position_counts.values,
                     labels={'x': column, 'y': 'Count'})
        fig.update_layout(title_text=column)
        fig.show()